# 신경망의 활성화함수,행렬곱 계층 구현하기
- 신경망을 구성하는 계층, 다양한 기능을 하는 계층을 각각 클래스로 구현한다

### 활성화 함수 계층 구현하기
- ReLU 함수
- 노드 사이에 스칼라값이 흐르는 간단한 계층 구현

In [9]:
import numpy as np
# forward는 넘파이 배열을 인수로 받는다
# mask 변수에는 입력된 넘파이 배열에 bool 정보가 대신 들어가 있다.
class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self,x):
        self.mask = (x <= 0) # x 배열 내에 0보다 작은 원소에 대해 True 부여
        out = x.copy() 
        out[mask] = 0 # x 배열 내에 0보다 작은 원소에는 0을 부여한다
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0 # 입력값이 0보다 작은 경우는 역전파의 출력이 0, 0보다 큰 경우에는 역전파 그대로 출력
        return dout

### 활성화 함수 계층 구현하기
- Sigmoid 함수(간소화 버전)
- 노드 사이에 스칼라값이 흐르는 간단한 계층 구현

In [14]:
# 시그모이드 함수의 연산을 한번에 묶어서 간소화한 Sigmoid 함수의 계층을 구현할 수 있다

class Sigmoid_simple:
    def __init__(self):
        self.out = None # Sigmoid의 역전파는 순전파 출력값으로만 계산할 수 있다
        
    def forward(self,x):
        self.out = 1 / (1+np.exp(-x))
        return self.out
        
    def backward(self,dout):
        dx = dout*(self.out)*(1-self.out)
        return dx

### 활성화 함수 계층 구현하기
- Sigmoid 함수(간소화 버전)
- 시그모이드 함수의 연산을 계산 그래프로 하나하나 구현할 수 있다
- 기존의 사용했던 곱셈계층, 덧셈계층에 이어서 나눗셈계층, exp계층을 추가할 수 있다.
- 나눗셈계층과 exp 계층 구현해보기
- 노드 사이에 스칼라값이 흐르는 간단한 계층 구현

In [56]:
class divisionLayer:
    def __init__(self):
        self.out = None # 나눗셈 계층의 역전파는 출력값 ** 2 에 - 를 붙인 것
    
    def forward(self,x):
        if x != 0: # zero division error 방지
            self.out = 1/x
            return self.out
    
    def backward(self,dout):
        dx = dout *  (- (self.out)**2)
        return dx

In [57]:
dv_layer = divisionLayer()
exp = dv_layer.forward(1)
print(dv_layer.backward(2))

-2.0


In [58]:
class expLayer:
    def __init__(self):
        self.out = None # exp 계층의 역전파는 순전파 출력값을 그대로 곱한 것
        
    def forward(self,x):
        self.out = np.exp(x)
        return self.out
    
    def backward(self,dout):
        dx = dout * self.out
        return dx

In [59]:
exp_layer = expLayer()
exp_layer.forward(1)
exp_layer.backward(2)

5.43656365691809

### Affine/Softmax 함수 계층 구현하기
- 노드 사이에서 스칼라값이 흐르던 앞선 구현과 달리, 노드 사이에 행렬이 흐른다
- 역전파 구현시, 행렬의 형상에 주의해야 한다. 행렬의 곱에서는 대응하는 차원의 원소수가 일치해야하기 때문이다

In [67]:
# 입력데이터 X가 1차원 배열인 경우
X = np.random.rand(2)
print("X : ",X)
W = np.random.rand(2,3)
print("W : ",W)
B = np.random.rand(3)
print("B : ",B)

Y = np.dot(X,W) + B
Y

X :  [0.26831188 0.43885052]
W :  [[0.44697763 0.79231776 0.43770828]
 [0.97050727 0.62366689 0.13089167]]
B :  [0.68460115 0.00747681 0.04261928]


array([1.23043817, 0.49376162, 0.21750349])

In [72]:
# 입력데이터 X가 2차원 배열인 경우
X = np.random.rand(2,2)
print("X : ",X)
W = np.random.rand(2,3)
print("W : ",W)
B = np.random.rand(3)
print("B : ",B)

Y = np.dot(X,W) + B
Y

X :  [[0.45828216 0.38546584]
 [0.16666418 0.49801699]]
W :  [[0.08067526 0.67812068 0.74333135]
 [0.71770158 0.05247116 0.56744003]]
B :  [0.03896819 0.15032666 0.25130746]


array([[0.35258967, 0.48132311, 0.8106917 ],
       [0.40984145, 0.28947662, 0.65778895]])

### Affine 함수 계층 구현하기
- 2차원 배열이 흐르는 경우 행렬곱 계층의 구현

In [73]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        
        return out # 출력값이 저장될 필요가 없기 때문에 out은 보관될 필요가 없다
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.db = np.sum(dout + b,axis=0)
        self.dW = np.dot(self.x.T,dout)
        return dx